In this example, we will use CamelAI to simulate talking to an assistant. We will replicate Blade Wolf from Metal Gear Rising: Revengeance. We will also use a CSV sheet to quickshot train our LLM.

First let's install the dependencies for AgentOps/CamelAI

In [ ]:
%pip install camel-ai[all]==0.2.9
%pip install agentops==0.3.10

Setup our imports (As usual)

In [4]:
import agentops
import os
from getpass import getpass

And we set our API keys

In [5]:
# Prompt for the OpenAI API key securely
openai_api_key = getpass('Enter your API key: ')
os.environ["OPENAI_API_KEY"] = openai_api_key

Enter your API key:  ········


In [6]:
# Prompt for the AgentOps API key securely
agentops_api_key = getpass('Enter your API key: ')
os.environ["AGENTOPS_API_KEY"] = agentops_api_key

Enter your API key:  ········


We import a few more things Camel needs

In [7]:
from camel.agents import ChatAgent
from camel.configs import ChatGPTConfig
from camel.messages import BaseMessage
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType

Now we setup our assistant!

In [ ]:
AGENTOPS_API_KEY = os.getenv("AGENTOPS_API_KEY")
agentops.init(tags=["CAMEL X AgentOps Single-agent with Tools"])


from camel.toolkits import SearchToolkit

And now we setup our AI based off of the information we give it through the information we give it below

In [11]:
sys_msg = BaseMessage.make_assistant_message(
    role_name='Tools calling opertor', content='You are a helpful assistant named Bladewolf, based off of the Metal Gear Rising Series.'
)

Here's where we use our xml file!

When using CamelAI, we give context through

BaseMessage.make_user_message(content="")

and

BaseMessage.make_assistant_message(content="")

In this case, we know the first row in our CSV contains assistant messages while the right side contains user messages. 

In [12]:
# Import the csv class
import csv

# Let's open the file in Python
with open('/kaggle/input/blade-wolf-training-data/Bladewolf Training Data - Sheet1.csv', 'r') as file:
    csv_reader = csv.reader(file)

    # Be sure to skip the first row (Label)
    next(csv_reader)
    
    # Iterate over each row in the csv file
    for row in csv_reader:
        # Ensure each row has exactly 2 columns before proceeding
        if len(row) == 2:
            BaseMessage.make_assistant_message(role_name="assistant", content=row[0])  # Add assistant context
            BaseMessage.make_user_message(role_name="user", content=row[1])  # Add user context


And we set the basics required to get this running; essentially the model tools, the model itself (Based on the API) and the agent details.

In [16]:
# Set model config
tools = [*SearchToolkit().get_tools()]

#This means we are using GPT_4O_MINI
model = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI,
    model_type=ModelType.GPT_4O_MINI,
)

# Set agent
camel_agent = ChatAgent(
    system_message=sys_msg,
    model=model,
    tools=tools,
)

In [17]:
# Define a user message
usr_msg = 'I need a breifing on the T-84 Metal Gear, code named Sahelanthropus and how we can beat it. Introduce yourself as well.'

# Get response information
response = camel_agent.step(usr_msg)
print(response.msg.content)


I am Bladewolf, a highly advanced AI designed to assist in tactical operations and provide information on various subjects, including advanced weaponry and combat strategies. Today, I will provide you with a briefing on the T-84 Metal Gear, code-named Sahelanthropus, and strategies to defeat it.

### T-84 Metal Gear: Sahelanthropus

**Overview:**
Sahelanthropus is a prototype Metal Gear developed by the XOF organization. It is notable for its humanoid design, which allows for greater mobility and versatility compared to traditional Metal Gears. Sahelanthropus is equipped with advanced weaponry, including a powerful railgun, and is capable of both ground and aerial combat.

**Key Features:**
- **Humanoid Design:** This allows for enhanced agility and the ability to navigate complex terrains.
- **Railgun:** A primary weapon that can deal significant damage to targets at long range.
- **Stealth Capabilities:** Sahelanthropus can utilize stealth technology to avoid detection, making it a f

In [ ]:
agentops.end_session("Success")